## Код для предобработки текстов приговоров.

Чтобы импортировать функции из этой тетрадки(и из любой другой), нужно установить import-ipynb:

In [1]:
!pip install import-ipynb

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Затем важно, чтобы все нужные тетрадки лежали в одной директории. Потом запускаем такое: 

In [1]:
import import_ipynb

Готово! Теперь функции из этой тетрадки доступны в вашей тетрадке. Вы восхитительны!

Приговоры в формате xml, чтобы работать с данными, нам нужно их распарсить.

In [2]:
import re
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [12]:
def to_bound_pattern(patterns):
    return re.compile('(?:{}).*\\n'.format('|'.join('\\s*'.join(x for x in s) for s in patterns)), re.IGNORECASE)

begin_pattern = to_bound_pattern(["УСТАНОВИЛ"])
end_pattern = to_bound_pattern(["ПРИГОВОРИЛ", "ПОСТАНОВИЛ"])

def parse_xml(path):  # парсим xml

    meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '',
                 'category': '', 'punishment_type': '', 'punishment_term': ''}  # складываем мета-данные в словарь

    with open(path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup("<soup>" + f.read() + "</soup>", 'xml')

        meta_data['court'] = soup.court.string  # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string

        html_soup = BeautifulSoup(soup.body.string, 'lxml')

        return re.sub('\\n+', '\\n ', html_soup.get_text()), meta_data


def get_parts(text):  # делим на части
    
    begin, main_part = re.split(begin_pattern, text, 1)
    main_part, end = re.split(end_pattern, main_part, 1)
    
    return begin, main_part, end


def clean(text):
    pattern1 = re.compile(r'(п|ч|ст)(\.|\s|\d)')
    pattern2 = re.compile(r'((У|Г)П?К|КоАП|ПДД)\sРФ')
    pattern3 = re.compile('\d')
    pattern4 = re.compile('(ДД.ММ.ГГ|дд.мм.гг)')

    text = re.sub(pattern1, ' ABBR ', text)
    text = re.sub(pattern2, ' DOCUMENT ', text)
    text = re.sub(pattern3, ' DIGIT ', text)
    text = re.sub(pattern4, ' DATE ', text)
    text = text.replace('\xad', '')

    return text

In [5]:
def normalize(text):
    punct = punctuation+'«»—…“”*№–'
    morph = MorphAnalyzer()
    words = [word.strip(punct) for word in text.lower().split()] 
    words = [word for word in words if word]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [6]:
# Убрать в отдельный модуль с заранее подготовленной train_data, и чтобы принимал только text
def segmentate(train_data:str, text: str): #очень желательно, чтобы тренировочная выборка отличалась от той, на которую ее применяют.
    
    trainer = PunktTrainer()
    trainer.INCLUDE_ALL_COLLOCS = True
    trainer.train(train_data) #на случай, если тренировочные данные в формате списка, добавить ('\n'.join(train_data))
    tokenizer = PunktSentenceTokenizer(trainer.get_params())
    
    return tokenizer.tokenize(text)

In [11]:
def preprocess(path):
    text, meta_data = parse_xml(path)
    return tuple(map(clean, get_parts(text))), meta_data